# Where is the International Space Station?

The International Space Station (ISS) is the largest human-made body in low Earth orbit. It has been inhabited continously since November 2000 and NASA is considering allowing tourists to visit it!

## Data

[Open Notify](http://open-notify.org/Open-Notify-API/) is a collection of NASA and space APIs for public use, developped by [Nathan Bergey](https://github.com/natronics).

The [GeoNames geographical database](http://www.geonames.org) covers all countries and contains over eleven million placenames that can be downloaded or accessed via their web services.

## Project

Check out [my project here](https://fannykassapian.github.io/iss.html).

In [17]:
# Import libraries
import numpy as np
import pandas as pd
import datetime as dt
import json
import requests

# Who's up there?

API:
http://open-notify.org/Open-Notify-API/People-In-Space/

In [18]:
response_1 = requests.get('http://api.open-notify.org/astros.json')
response_1 = response_1.json()
response_1

{'message': 'success',
 'number': 6,
 'people': [{'craft': 'ISS', 'name': 'Oleg Kononenko'},
  {'craft': 'ISS', 'name': 'David Saint-Jacques'},
  {'craft': 'ISS', 'name': 'Anne McClain'},
  {'craft': 'ISS', 'name': 'Alexey Ovchinin'},
  {'craft': 'ISS', 'name': 'Nick Hague'},
  {'craft': 'ISS', 'name': 'Christina Koch'}]}

In [19]:
number_people = response_1['number']
number_people

6

In [20]:
people = [i['name'] for i in response_1['people']]
people

['Oleg Kononenko',
 'David Saint-Jacques',
 'Anne McClain',
 'Alexey Ovchinin',
 'Nick Hague',
 'Christina Koch']

# Where is the ISS?

API:
http://open-notify.org/Open-Notify-API/ISS-Location-Now/

## Position

In [21]:
response_2 = requests.get('http://api.open-notify.org/iss-now.json')
response_2 = response_2.json()
response_2

{'iss_position': {'latitude': '-18.4164', 'longitude': '56.8185'},
 'message': 'success',
 'timestamp': 1560530272}

In [22]:
lat = float(response_2['iss_position']['latitude'])
long = float(response_2['iss_position']['longitude'])
(lat, long)

(-18.4164, 56.8185)

## Time

In [23]:
# Unix timestamp
timestamp = response_2['timestamp']
timestamp

1560530272

In [24]:
# Datetime object
time_dt = dt.datetime.fromtimestamp(timestamp)
time_dt

datetime.datetime(2019, 6, 14, 18, 37, 52)

In [25]:
when = time_dt.strftime('%Y-%m-%d %H:%M:%S')
when

'2019-06-14 18:37:52'

In [32]:
hour = time_dt.strftime('%H:%M:%S')
hour

'18:37:52'

# Map

In [26]:
from ipyleaflet import Map, basemaps, basemap_to_tiles, Marker

center = (lat,long)

m = Map(
    center=center,
    zoom=3
)

# Marker
marker = Marker(location=center, draggable=False)
m.add_layer(marker);

# Add TileLayer
hour = int(time_dt.strftime('%H'))

layer = basemaps.NASAGIBS.ViirsTrueColorCR

style_layer = basemap_to_tiles(layer) 
m.add_layer(style_layer)

m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a> contributors'}, center=[-18.4164, 56.8185], layers=(TileLayer(base=True, max_zoom=19, min_zoom=1, options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'tile_size']), Marker(draggable=False, location=[-18.4164, 56.8185], options=['alt', 'draggable', 'keyboard', 'rise_offset', 'rise_on_hover', 'rotation_angle', 'rotation_origin', 'title', 'z_index_offset']), TileLayer(attribution='Imagery provided by services from the Global Imagery Browse Services (GIBS), operated by the NASA/GSFC/Earth Science Data and Information System (<a href="https://earthdata.nasa.gov">ESDIS</a>) with funding provided by NASA/HQ.', max_zoom=9, min_zoom=1, name='NASAGIBS.ViirsTrueColorCR', options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'tile_size'], url='https://gibs.earthdata.nasa.gov/wmts/epsg3857/best/VIIRS_SNPP_CorrectedReflectance_TrueColor/default/2019-06-13/GoogleMapsCompatible_Level9/{z}/{y}/{x}.jpg')), options=['attribution_control', 'basemap', 'bounce_at_zoom_limits', 'box_zoom', 'center', 'close_popup_on_click', 'double_click_zoom', 'dragging', 'inertia', 'inertia_deceleration', 'inertia_max_speed', 'interpolation', 'keyboard', 'keyboard_pan_offset', 'keyboard_zoom_offset', 'max_zoom', 'min_zoom', 'scroll_wheel_zoom', 'tap', 'tap_tolerance', 'touch_zoom', 'world_copy_jump', 'zoom', 'zoom_animation_threshold', 'zoom_control', 'zoom_start'], zoom=3)

# Country

The GeoNames geographical database covers all countries and contains over eleven million placenames that can be downloaded or accessed via their [web services](https://www.geonames.org/export/web-services.html).

*Important: Do not use the 'demo' account for your app or your tests. The cell below was actually ran with my user name before I changed it.*

If the latitude and longitude returned by Open Notify correspond to a country, we return the name of that country using the countryCode API, otherwise, we return the name of the ocean/sea using the ocean API:

- countryCode API : http://api.geonames.org/countryCode?
- ocean API: http://api.geonames.org/ocean?

In [31]:
parameters = {'lat': lat, 'lng': long, 'type': 'JSON', 'radius':1, 'username':'demo'}
response_3 = requests.get('http://api.geonames.org/countryCode?', params = parameters)
response_3 = response_3.json()

if 'countryName' in response_3:
    country = response_3['countryName']
    print(country)
else:
    parameters = {'lat': lat, 'lng': long, 'radius':1, 'username':'demo'};
    response_4 = requests.get('http://api.geonames.org/oceanJSON?', params = parameters)
    response_4 = response_4.json()
    ocean = response_4['ocean']['name']
    print(ocean)

Indian Ocean
